In [21]:
import pandas as pd
import sys
import Levenshtein
from Levenshtein import *

In [22]:
def readedits(fileerror, filecorrect, id_from, id_to):
    data = []
    with open(fileerror, 'r') as erf:
        with open(filecorrect, 'r') as cor:
            i = 0
            while i < id_to:
                if i >= id_from:
                    data.append([erf.readline(), cor.readline()])
                else:
                    erf.readline()
                    cof.readline()
                i +=1
    return data

In [23]:
data = readedits('wiked.tok.err', 'wiked.tok.cor', 0, 1000)

In [24]:
data[150]

['Radical reactions dont deal with charged particles but with radicals .\n',
 'Radical reactions don &apos;t deal with charged particles but with radicals .\n']

In [34]:
def get_statistis_words(data, max_distance=3):
    res_words = []
    for i, line in enumerate(data):
        errsent = line[0].split()
        corsent = line[1].split()
        for errw in errsent:
            for corw in corsent:
                d = distance(errw, corw)
                if (d > 0) and d < max_distance:
                    res_words.append([errw, corw])
    return res_words

In [35]:
pairs = get_statistis_words(data)

In [38]:
pairs

[['-', 'to'],
 ['-', 'be'],
 ['-', 'to'],
 ['-', 'be'],
 ['-', 'to'],
 ['-', 'be'],
 ['-', 'to'],
 ['-', 'be'],
 ['How', 'to'],
 ['to', '-'],
 ['to', '-'],
 ['to', '-'],
 ['to', '-'],
 ['to', 'How'],
 ['to', 'be'],
 ['be', '-'],
 ['be', '-'],
 ['be', '-'],
 ['be', '-'],
 ['be', 'to'],
 ['The', 'the'],
 ['The', 'the'],
 ['Wikimedia', 'Wikipedia'],
 ['Textbook', 'textbooks'],
 ['is', 'a'],
 ['is', 'of'],
 ['is', 'up'],
 ['is', 'on'],
 ['is', '10'],
 ['is', ','],
 ['is', 'of'],
 ['is', 'of'],
 ['is', '.'],
 ['a', 'is'],
 ['a', 'of'],
 ['a', 'up'],
 ['a', 'on'],
 ['a', '10'],
 ['a', ','],
 ['a', 'of'],
 ['a', 'of'],
 ['a', 'and'],
 ['a', '.'],
 ['of', 'is'],
 ['of', 'a'],
 ['of', 'up'],
 ['of', 'on'],
 ['of', '10'],
 ['of', ','],
 ['of', 'for'],
 ['of', '.'],
 ['the', 'The'],
 ['Wikipedia', 'Wikimedia'],
 ['up', 'is'],
 ['up', 'a'],
 ['up', 'of'],
 ['up', 'on'],
 ['up', '10'],
 ['up', ','],
 ['up', 'of'],
 ['up', 'of'],
 ['up', '.'],
 ['for', 'of'],
 ['for', 'on'],
 ['for', 'of'],
 ['for',